In [7]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob 

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\\delete\\European Chemicals Agency'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [4]:
def handle_popup():
    try:
        driver.find_elements(by=By.XPATH, value='//*[@id="_viewsubstances_WAR_echarevsubstanceportlet_acceptDisclaimerButton"]/span')[0].click()
    except:
        pass
    return None

In [6]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\ECHA"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("http://echa.europa.eu/nl/ed-assessment")
driver.implicitly_wait(5)
handle_popup()
driver.find_element(by=By.XPATH, value='//*[@id="_disslists_WAR_disslistsportlet_exportButtonXLS"]/span').click()
time.sleep(10)

driver.close()

In [26]:
path = 'Y:\\delete\\ECHA'
filenames = glob.glob(path + "\*.xlsx")
for f in filenames:
    dataset = pd.read_excel(f,skiprows=3)
dataset = dataset[['CAS no','Status']]
dataset.rename(columns = {'CAS no':'CAS Number'}, inplace = True)
dataset = dataset.loc[dataset['CAS Number'] != '-']
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)

CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Concluded'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.Status.fillna(result.value, inplace=True)
del result['value']
result.head(2)

d:\python3.9\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,CAS Number,Status
0,100-21-0,Concluded
1,77-90-7,Concluded


In [29]:
result['Jurisdiction'] = 'Europe'
result['RegulatoryAgency'] = 'European Chemicals Agency (ECHA)'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Endocrine Disruption'

result['Description'] = 'Under investigation'
result.loc[result['Status'] == 'Concluded', 'Description'] = 'Not an endocrine disuptor'

result['HazardClassification'] = 'High'
result.loc[result['Status'] == 'Concluded', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'Under development, under development under SEV, postponed'
result.loc[result['Status'] == 'Concluded', 'CauseOfClassification'] = 'Not ED'

result['SourceName'] = 'European Chemicals Agency'
result['SourceURL'] = 'http://echa.europa.eu/nl/ed-assessment'
result = result.drop(['Status'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,100-21-0,Europe,European Chemicals Agency (ECHA),Health,Endocrine Disruption,Not an endocrine disuptor,Low,Not ED,European Chemicals Agency,http://echa.europa.eu/nl/ed-assessment
1,77-90-7,Europe,European Chemicals Agency (ECHA),Health,Endocrine Disruption,Not an endocrine disuptor,Low,Not ED,European Chemicals Agency,http://echa.europa.eu/nl/ed-assessment


In [ ]:
results.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [11]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:05:57.446392
